In [1]:
# %% test cuda
import torch #type: ignore
torch.cuda.is_available()

True

In [1]:
# %% test a specific model
from src import ModelTrainer
ModelTrainer.fit(process = 2 , resume = 0 , checkname= 1)

Use device name: NVIDIA GeForce RTX 4090


24-04-18 23:11:00|MOD:ModelModule |: Model Specifics:
24-04-18 23:11:00|MOD:ModelModule |: Start Process [Load Data]!


--Process Queue : Data + Test
--Model_name is set to patch_tst_day_ShortTest!
{'random_seed': None,
 'model_name': 'patch_tst_day_ShortTest',
 'model_module': 'patch_tst',
 'model_data_type': 'day',
 'model_data_prenorm': {'day': {'divlast': True, 'histnorm': True}},
 'labels': ['std_lag1_10'],
 'beg_date': 20170103,
 'end_date': 20171201,
 'interval': 120,
 'input_step_day': 5,
 'sample_method': 'train_shuffle',
 'shuffle_option': 'epoch'}
{'hidden_dim': [32],
 'seqlens': [{'day': 30, '30m': 30, 'dms': 30}],
 'seq_len': [30],
 'dropout': [0.1],
 'attn_dropout': [0.1],
 'num_output': [1],
 'shared_embedding': [True],
 'shared_head': [True],
 'revin': [True],
 'act_type': ['gelu'],
 'patch_len': [5],
 'stride': [2],
 'n_layers': [2],
 'n_heads': [16],
 'd_ff': [64]}
use /home/mengkjin/Workspace/learndl/data/torch_pack/day.20240313.pt


24-04-18 23:11:04|MOD:ModelModule |: Finish Process [Load Data]! Cost 3.2Secs
24-04-18 23:11:04|MOD:ModelModule |: Start Process [Test Model]!
24-04-18 23:11:04|MOD:ModelModule |: Each Model Date Testing Mean Score(spearman):
Models            0       0       0
Output         best swalast swabest


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
score function of [spearman] calculated and success!


20170103     0.0748  0.0552  0.0723
20170704     0.1015  0.1020  0.1032
AllTimeAvg   0.0871  0.0768  0.0866
AllTimeSum    19.43   17.13   19.30
Std          0.0691  0.0816  0.0708
TValue        18.82   14.06   18.25
AnnIR        6.1748  4.6115  5.9873
24-04-18 23:11:16|MOD:ModelModule |: Finish Process [Test Model]! Cost 11.9 Secs


In [3]:
import torch
from torch import Tensor , nn

head_dim = 64
nn.Parameter(Tensor(head_dim ** -0.5), requires_grad=True)

TypeError: new(): data must be a sequence (got float)

In [6]:
torch.tensor(head_dim ** -0.5)

tensor(0.1250)

In [1]:
# prepare data
from src import DataModule
DataModule.prepare_data()

Use device name: NVIDIA GeForce RTX 4090


24-03-31 09:52:11|MOD:data_preprocessing|: if_train is False , Data Processing start!
24-03-31 09:52:11|MOD:data_preprocessing|: 3 datas :['y', 'trade_day', 'trade_30m']


Sun Mar 31 09:52:11 2024 : y start ...
labels blocks reading ret10_lag DataBase's ...... cost 0.39 secs
labels blocks reading ret20_lag DataBase's ...... cost 0.33 secs
labels blocks merging ...... cost 0.05 secs
models blocks reading risk_exp DataBase's ...... cost 1.66 secs
models blocks merging ...... cost 0.17 secs
y blocks process ...... cost 0.88 secs
y blocks masking ...... cost 0.05 secs
y blocks saving  ...... cost 0.14 secs
y blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:14 2024 : y finished! Cost 3.67 Seconds
Sun Mar 31 09:52:14 2024 : trade_day start ...
trade blocks reading day DataBase's ...... cost 0.56 secs
trade_day blocks merging ...... cost 0.00 secs
trade_day blocks process ...... cost 0.03 secs
trade_day blocks masking ...... cost 0.05 secs
trade_day blocks saving  ...... cost 0.16 secs
trade_day blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:15 2024 : trade_day finished! Cost 0.81 Seconds
Sun Mar 31 09:52:15 2024 : trade_30m start ...
trade blocks r

24-03-31 09:52:21|MOD:data_preprocessing|: Data Processing Finished! Cost 10.02 Seconds


... cost 1.13 secs
trade_30m blocks norming ...... cost 0.00 secs
Sun Mar 31 09:52:21 2024 : trade_30m finished! Cost 5.19 Seconds
